In [18]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [19]:
#This will create a Collection/Database called department if it doesn't already exist

coll_department = client['department']

department = coll_department.department

In [20]:
department.insert_many([{"dep_name": "IT", "DepartmentHead":"Jason"},
                            {"dep_name": "Admin", "DepartmentHead":"Nial"},
                            {"dep_name": "Accounts", "DepartmentHead":"Harris"}])


In [21]:
#This will create a Collection/Database called employees if it doesn't already exist

import pandas as pd

db = client['employees']

df_employees = pd.DataFrame(list(db.employees.find()))


In [22]:
df_employees.head()

,_id,name,Department,Salary
0,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000
1,5f5852e11bb7b32b0ab9e874,Joseph,IT,7000
2,5f5852e11bb7b32b0ab9e875,Alex,Accounts,5000
3,5f5852e11bb7b32b0ab9e876,Julie,IT,3000
4,5f5852e11bb7b32b0ab9e877,James,Admin,8000


In [24]:
df_department = pd.DataFrame(list(department.find()))

In [25]:
df_department.head()


,_id,dep_name,DepartmentHead
0,5f5a857ba249d17d3fb19c7b,IT,Jason
1,5f5a857ba249d17d3fb19c7c,Admin,Nial
2,5f5a857ba249d17d3fb19c7d,Accounts,Harris
3,5f5a8a8a686b40444782a309,IT,Jason
4,5f5a8a8a686b40444782a30a,Admin,Nial


### Left Join: 

Left join uses only keys from left frame, similar to a SQL left outer join

In [26]:
df_employees.merge(df_department,left_on="Department",right_on="dep_name",how="left")

,_id_x,name,Department,Salary,_id_y,dep_name,DepartmentHead
0,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5a857ba249d17d3fb19c7b,IT,Jason
1,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5a8a8a686b40444782a309,IT,Jason
2,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5bcc504d4462a3761e61f9,IT,Jason
3,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5fa38c561313207d47ab3d,IT,Jason
4,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5fa393561313207d47ab41,IT,Jason
...,...,...,...,...,...,...,...
100,5f5a5d28880c9e07e8eb8239,Kevin,IT,6500,5f5a857ba249d17d3fb19c7b,IT,Jason
101,5f5a5d28880c9e07e8eb8239,Kevin,IT,6500,5f5a8a8a686b40444782a309,IT,Jason
102,5f5a5d28880c9e07e8eb8239,Kevin,IT,6500,5f5bcc504d4462a3761e61f9,IT,Jason
103,5f5a5d28880c9e07e8eb8239,Kevin,IT,6500,5f5fa38c561313207d47ab3d,IT,Jason


### Inner Join: 

Use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

In [27]:
df_employees.merge(df_department,left_on="Department",right_on="dep_name",how="inner")

,_id_x,name,Department,Salary,_id_y,dep_name,DepartmentHead
0,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5a857ba249d17d3fb19c7b,IT,Jason
1,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5a8a8a686b40444782a309,IT,Jason
2,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5bcc504d4462a3761e61f9,IT,Jason
3,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5fa38c561313207d47ab3d,IT,Jason
4,5f5852e11bb7b32b0ab9e873,Jessica,IT,6000,5f5fa393561313207d47ab41,IT,Jason
...,...,...,...,...,...,...,...
100,5f5a5d28880c9e07e8eb8238,Jacob,Admin,9000,5f5a857ba249d17d3fb19c7c,Admin,Nial
101,5f5a5d28880c9e07e8eb8238,Jacob,Admin,9000,5f5a8a8a686b40444782a30a,Admin,Nial
102,5f5a5d28880c9e07e8eb8238,Jacob,Admin,9000,5f5bcc504d4462a3761e61fa,Admin,Nial
103,5f5a5d28880c9e07e8eb8238,Jacob,Admin,9000,5f5fa38c561313207d47ab3e,Admin,Nial


### To get the total number of records in the collection

In [28]:
pipeline = [
     {"$group": {"_id": None,"Count": {"$sum": 1}}}]
grp_employees = db.employees.aggregate(pipeline)


In [29]:
for employee in grp_employees:
  print(employee)

{'_id': None, 'Count': 21}


### To group by department and get the total salary for each department

In [30]:
pipeline = [
     {"$group": {"_id": "$Department","Salary": {"$sum": "$Salary"}}}]
grp_employees = db.employees.aggregate(pipeline)


In [31]:
for employee in grp_employees:
  print(employee)

{'_id': 'IT', 'Salary': 67500}
{'_id': 'Accounts', 'Salary': 15000}
{'_id': 'Admin', 'Salary': 51000}


### To group by department and get the average salary for each department

In [32]:
pipeline = [
     {"$group": {"_id": "$Department","Salary": {"$avg": "$Salary"}}}]
grp_employees = db.employees.aggregate(pipeline)


In [35]:
for employee in grp_employees:
  print(employee['_id'],"\t Average Salary ",employee['Salary'])

### To get the employees with the lowest salary

In [36]:
pipeline = [
     {"$group": {"_id": None ,"Minimum Salary": {"$min": "$Salary"}}}]
grp_employees = db.employees.aggregate(pipeline)


In [37]:
# for employee in grp_employees:
#     for spec_emp in employee.find({"Salary":employee['Minimum Salary']}):
#         print("Name ",spec_emp['name'],"\nSalary ",spec_emp['Salary'])


for employee in grp_employees:
    for spec_emp in db.employees.find({"Salary":employee['Minimum Salary']}):
        print("Name ",spec_emp['name'],"\nSalary ",spec_emp['Salary'])


Name  Julie 
Salary  3000
Name  Julie 
Salary  3000
Name  Julie 
Salary  3000
